# Parse Experimental Results & Generate Latex Tables

<a id='toc'/>

[Table of Contents](#toc)
1. [Distribution of POI Visit Duration](#1.-Distribution-of-POI-Visit-Duration)
1. [Distribution of POI Popularity & #Visits](#2.-Distribution-of-POI-Popularity-&-#Visits)
1. [POI Categories](#3.-POI-Categories)
1. [Latex Table for Dataset Statistics](#4.-Latex-Table-for-Dataset-Statistics)
1. [Latex Table for Recommendation Results](#5.-Latex-Table-for-Recommendation-Results)

In [ ]:
% matplotlib inline
import os, pickle, types
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
data_ijcai = 'data/data-ijcai15'
#data_recsys = 'data/data-recsys16'
data_recsys = 'data/data-cikm16'

In [ ]:
dat_suffix = ['Edin', 'Glas', 'Melb', 'Osak', 'Toro']
dat_names = ['Edinburgh', 'Glasgow', 'Melbourne', 'Osaka', 'Toronto']

In [ ]:
dat_types = ['all']

In [ ]:
methods_all = ['\\textsc{Random}', '\\textsc{PersTour}\cite{ijcai15}', '\\textsc{PersTour-L}', \
               '\\textsc{PoiPopularity}', '\\textsc{PoiRank}', '\\textsc{Markov}', '\\textsc{MarkovPath}', \
               '\\textsc{Rank+Markov}', '\\textsc{Rank+MarkovPath}']

In [ ]:
num_photos = [82060, 29019, 94142, 392420, 157505]

## 1. Distribution of POI Visit Duration

In [ ]:
fig = plt.figure(figsize=[15, 2])
for dat_ix in range(len(dat_suffix)):
    ftraj = os.path.join(data_recsys, 'traj-noloop-all-' + dat_suffix[dat_ix] + '.csv')
    traj_all = pd.read_csv(ftraj)
    plt.subplot(1, len(dat_suffix), dat_ix+1)
    ax = traj_all['poiDuration'].hist(bins=20)
    ax.set_xscale('log')
    ax.set_yscale('log')
    #ax.set_xlim(left=1)
    if dat_ix == len(dat_suffix)-2:
        ax.set_xlim(left=500)
    if dat_ix == len(dat_suffix)-1:
        ax.set_xlim(left=2000)
    
    ax.set_title(dat_names[dat_ix])
    if dat_ix == 2:
        ax.set_xlabel('POI Visit Duration (seconds)')
    if dat_ix == 0:
        ax.set_ylabel('#POI Visits')
pdf = PdfPages('visit_duration.pdf')
pdf.savefig(fig)
pdf.close()

## 2. Distribution of POI Popularity & #Visits

Compute POI properties, e.g., popularity, total number of visit, average visit duration.

In [ ]:
def calc_poi_info(trajid_list, traj_all, poi_all):
    assert(len(trajid_list) > 0)
    # to allow duplicated trajid
    poi_info = traj_all[traj_all['trajID'] == trajid_list[0]][['poiID', 'poiDuration']].copy() 
    for i in range(1, len(trajid_list)):
        traj = traj_all[traj_all['trajID'] == trajid_list[i]][['poiID', 'poiDuration']]
        poi_info = poi_info.append(traj, ignore_index=True)
    
    poi_info = poi_info.groupby('poiID').agg([np.mean, np.size])
    poi_info.columns = poi_info.columns.droplevel()
    poi_info.reset_index(inplace=True)
    poi_info.rename(columns={'mean':'avgDuration', 'size':'nVisit'}, inplace=True)
    poi_info.set_index('poiID', inplace=True) 
    poi_info['poiCat'] = poi_all.loc[poi_info.index, 'poiCat']
    poi_info['poiLon'] = poi_all.loc[poi_info.index, 'poiLon']
    poi_info['poiLat'] = poi_all.loc[poi_info.index, 'poiLat']
    
    # POI popularity: the number of distinct users that visited the POI
    pop_df = traj_all[traj_all['trajID'].isin(trajid_list)][['poiID', 'userID']].copy()
    pop_df = pop_df.groupby('poiID').agg(pd.Series.nunique)
    pop_df.rename(columns={'userID':'nunique'}, inplace=True)
    poi_info['popularity'] = pop_df.loc[poi_info.index, 'nunique']
    
    return poi_info.copy()

In [ ]:
fig = plt.figure(figsize=[15, 2])
for dat_ix in range(len(dat_suffix)):
    fpoi = os.path.join(data_recsys, 'poi-' + dat_suffix[dat_ix] + '.csv')
    ftraj = os.path.join(data_recsys, 'traj-noloop-all-' + dat_suffix[dat_ix] + '.csv')
    poi_all = pd.read_csv(fpoi)
    poi_all.set_index('poiID', inplace=True)
    traj_all = pd.read_csv(ftraj)
    poi_info = calc_poi_info(traj_all['trajID'].unique(), traj_all, poi_all)
    plt.subplot(1, len(dat_suffix), dat_ix+1)
    #ax = poi_info['popularity'].hist(bins=8)
    ax = poi_info['nVisit'].hist(bins=8)
    
    ax.set_title(dat_names[dat_ix])
    if dat_ix == 2:
        #ax.set_xlabel('POI Popularity')
        ax.set_xlabel('#Visits at POI')
    if dat_ix == 0:
        ax.set_ylabel('#POIs')
    labels = ax.get_xticklabels()
    plt.setp(labels, rotation=30, fontsize=10)
#pdf = PdfPages('poi_popularity.pdf')
pdf = PdfPages('poi_nvisit.pdf')
pdf.savefig(fig)
pdf.close()

## 3. POI Categories

In [ ]:
cat_set = set()
for dat_ix in range(len(dat_suffix)):
    fpoi = os.path.join(data_recsys, 'poi-' + dat_suffix[dat_ix] + '.csv')
    ftraj = os.path.join(data_recsys, 'traj-noloop-all-' + dat_suffix[dat_ix] + '.csv')
    poi_all = pd.read_csv(fpoi)
    traj_all = pd.read_csv(ftraj)
    poi_all.set_index('poiID', inplace=True)
    cats = sorted(poi_all.loc[traj_all['poiID'].unique(), 'poiCat'].unique())
    cat_set = cat_set | set(cats)
    cat_str = ''.join([x + ', ' for x in cats])
    print('%s: %s' % (dat_names[dat_ix], cat_str[:-2]))
print('\n%s' % sorted(cat_set))

## 4. Latex Table for Dataset Statistics

In [ ]:
strs = []
for dset in dat_types:
    if dset == 'all':     
        title = 'of all trajectories without loops'
        noloop = True
    if dset == 'nofew':   
        title = 'of users with more than 5 (including 5) trajectories with loops'
        noloop = False
    strs.append('\\begin{table*}\n')
    strs.append('\\centering\n')
    #strs.append('\\caption{Dataset ' + title + '}\n')
    #strs.append('\\small\n')
    strs.append('\\begin{tabular}{' + 'l' + 5*'r' + '} \\hline\n')
    #strs.append('\\textbf{City} & \\textbf{\\#POIs} & \\textbf{\\#Users} & ')
    #strs.append('\\textbf{\\#POI Visits} & \\textbf{\\#Trajectories} & \\textbf{\\#TotalNodes} \\\\ \\hline\n')
    strs.append('\\textbf{Dataset} & \\textbf{\\#Photos} & \\textbf{\\#POI Visits} & \\textbf{\\#Trajectories} & ')
    strs.append('\\textbf{\\#Users} \\\\ \\hline\n')
    
    for dat_ix in range(len(dat_names)):
        if noloop == True:
            ftraj = os.path.join(data_recsys, 'traj-noloop-' + dset + '-' + dat_suffix[dat_ix] + '.csv')
        else:
            ftraj = os.path.join(data_recsys, 'traj-' + dset + '-' + dat_suffix[dat_ix] + '.csv')
        traj_df = pd.read_csv(ftraj)
        total_nodes = traj_df[['trajID', 'trajLen']].copy().groupby('trajID').first().sum().values[0]
        strs.append(dat_names[dat_ix])
        #strs.append(' & ' + '{:,}'.format(traj_df['poiID'].unique().shape[0]))
        strs.append(' & ' + '{:,}'.format(num_photos[dat_ix]))
        strs.append(' & ' + '{:,}'.format(traj_df['#photo'].sum()))
        strs.append(' & ' + '{:,}'.format(traj_df['trajID'].unique().shape[0]))
        strs.append(' & ' + '{:,}'.format(traj_df['userID'].unique().shape[0]))
        #strs.append(' & ' + '{:,}'.format(total_nodes))
        strs.append(' \\\\ \n')
    strs.append('\\hline\n')
    strs.append('\\end{tabular}\n')
    strs.append('\\caption{Statistics of trajectory dataset}\n')
    strs.append('\\label{table:data}\n')
    strs.append('\\end{table*}\n\n')
print(''.join(strs))

## 5. Latex Table for Recommendation Results

Generate results filenames.

In [ ]:
def gen_fname(dat_ix, type_ix, noloop):
    assert(0 <= dat_ix < len(dat_suffix))
    assert(0 <= type_ix < len(dat_types))
    assert(isinstance(noloop, bool))
    
    if noloop == True:
        loop_str = 'noloop-'
    else:
        loop_str = ''
    
    type_str = dat_types[type_ix] + '-agnostic-'
    suffix = dat_suffix[dat_ix] + '.pkl'
    
    fname = loop_str + type_str
    frank = os.path.join(data_recsys, 'rank-' + fname + suffix)
    ftran = os.path.join(data_recsys, 'tran-' + fname + suffix)
    fcomb = os.path.join(data_recsys, 'comb-' + fname + '0_5-' + suffix)
    frand  = os.path.join(data_recsys, 'rand-' + fname + suffix)
    fijcai = os.path.join(data_ijcai, 'ijcai-' + dat_suffix[dat_ix] + '.pkl')
    return frank, ftran, fcomb, frand, fijcai

In [ ]:
gen_fname(1, 0, True)

Compute the F1 score for recommended trajectory.

In [ ]:
def calc_F1(traj_act, traj_rec, noloop=False):
    '''Compute recall, precision and F1 for recommended trajectories'''
    assert(isinstance(noloop, bool))
    assert(len(traj_act) > 0)
    assert(len(traj_rec) > 0)
    
    if noloop == True:
        intersize = len(set(traj_act) & set(traj_rec))
    else:
        match_tags = np.zeros(len(traj_act), dtype=np.bool)
        for poi in traj_rec:
            for j in range(len(traj_act)):
                if match_tags[j] == False and poi == traj_act[j]:
                    match_tags[j] = True
                    break
        intersize = np.nonzero(match_tags)[0].shape[0]
        
    recall = intersize / len(traj_act)
    precision = intersize / len(traj_rec)
    F1 = 2 * precision * recall / (precision + recall)
    return F1

In [ ]:
def calc_pairsF1(seq_act, seq_rec):
    assert(len(seq_act) > 0)
    assert(len(seq_act) == len(set(seq_act))) # no loops in seq_act
    n = len(seq_act)
    nr = len(seq_rec)
    n0 = n*(n-1) / 2
    n0r = nr*(nr-1) / 2
    
    # seq_act determines the correct visiting order
    order_df = pd.DataFrame(data=np.zeros((n, n), dtype=np.bool), columns=seq_act, index=seq_act)
    for i in range(n):
        poi1 = seq_act[i]
        for j in range(i+1, n):
            poi2 = seq_act[j]
            order_df.loc[poi1, poi2] = True
            
    nc = 0
    for i in range(nr):
        poi1 = seq_rec[i]
        for j in range(i+1, nr):
            poi2 = seq_rec[j]
            if poi1 in seq_act and poi2 in seq_act:
                if poi1 != poi2:
                    if order_df.loc[poi1, poi2] == True: nc += 1
    
    precision = nc / n0r
    recall = nc / n0
    if nc == 0:
        F1 = 0
    else:
        F1 = 2 * precision * recall / (precision + recall)
    return F1

Load results data.

In [ ]:
def load_results(dat_ix, type_ix):
    assert(0 <= dat_ix < len(dat_suffix))
    assert(0 <= type_ix < len(dat_types))
    
    noloop = True
    
    frank, ftran, fcomb, frand, fijcai = gen_fname(dat_ix, type_ix, noloop)
    #print(frank)
    assert(os.path.exists(frank))
    #print(ftran)
    assert(os.path.exists(ftran))
    #print(fcomb)
    assert(os.path.exists(fcomb))
    #print(frand)
    assert(os.path.exists(frand))
    #print(fijcai)
    assert(os.path.exists(fijcai))

    # load results data
    recdict_rank = pickle.load(open(frank, 'rb'))
    recdict_tran = pickle.load(open(ftran, 'rb'))
    recdict_comb = pickle.load(open(fcomb, 'rb'))
    recdict_rand = pickle.load(open(frand, 'rb'))
    recdict_ijcai = pickle.load(open(fijcai, 'rb'))
    
    return recdict_rank, recdict_tran, recdict_comb, recdict_rand, recdict_ijcai

Calculate F1-scores from loaded results.

In [ ]:
def calc_metrics(recdict_rank, recdict_tran, recdict_comb, recdict_rand, recdict_ijcai, func):
    assert(isinstance(func, types.FunctionType))
    
    # deal with missing values: 
    # get rid of recommendation that not all method are successful, 
    # due to ILP timeout, <start, end> of test doesn't exist in training set.
    assert(np.all(sorted(recdict_rank.keys()) == sorted(recdict_tran.keys())))
    assert(np.all(sorted(recdict_rank.keys()) == sorted(recdict_comb.keys())))
    
    keys_all = sorted(recdict_ijcai.keys() & recdict_rank.keys())
    
    # compute F1
    rank1 = []  # rank pop
    rank2 = []  # rank feature
    #for key in sorted(recdict_rank.keys()):
    for key in keys_all:
        rank1.append(func(recdict_rank[key]['REAL'], recdict_rank[key]['REC_POP']))
        rank2.append(func(recdict_rank[key]['REAL'], recdict_rank[key]['REC_FEATURE']))
    
    tran1 = []  # transition DP
    tran2 = []  # transition ILP
    #for key in sorted(recdict_tran.keys()):
    for key in keys_all:
        tran1.append(func(recdict_tran[key]['REAL'], recdict_tran[key]['REC_DP']))
        tran2.append(func(recdict_tran[key]['REAL'], recdict_tran[key]['REC_ILP']))

    comb1 = []  # combine rank and transition DP
    comb2 = []  # combine rank and transition ILP
    #for key in sorted(recdict_comb.keys()):
    for key in keys_all:
        comb1.append(func(recdict_comb[key]['REAL'], recdict_comb[key]['REC_DP']))
        comb2.append(func(recdict_comb[key]['REAL'], recdict_comb[key]['REC_ILP']))
            
    rand = []   # structured prediction
    #for key in sorted(recdict_rand.keys()):
    for key in keys_all:
        rand.append(func(recdict_rand[key]['REAL'], recdict_rand[key]['REC_RAND']))
    
    ijcai05T = []   # IJCAI method
    #ijcai10T = []   # IJCAI method
    ijcai05L = []   # IJCAI method
    #ijcai10L = []   # IJCAI method
    #for key in sorted(recdict_ijcai.keys()):
    for key in keys_all:
        #if func == calc_F1:
        #    ijcai05T.append(func(recdict_ijcai[key]['REAL'], recdict_ijcai[key]['REC05T']))
            #ijcai10T.append(func(recdict_ijcai[key]['REAL'], recdict_ijcai[key]['REC10T']))
        ijcai05T.append(func(recdict_ijcai[key]['REAL'], recdict_ijcai[key]['REC05T']))
        ijcai05L.append(func(recdict_ijcai[key]['REAL'], recdict_ijcai[key]['REC05L']))
        #ijcai10L.append(func(recdict_ijcai[key]['REAL'], recdict_ijcai[key]['REC10L']))        
    
    # compute mean and std of F1
    metrics = [rand]
    #if func == calc_F1:
    #    metrics = metrics + [ijcai05T]
    metrics = metrics + [ijcai05T]
    metrics = metrics + [ijcai05L, rank1, rank2, tran1, tran2, comb1, comb2]
    means = [np.mean(x) for x in metrics]
    stds  = [np.std(x)  for x in metrics]
    
    return means, stds

Generate Latex tables from calculated metrics.

In [ ]:
def gen_latex_table(mean_df, std_df, ismax_df, ismax2nd_df, type_ix, title, label):    
    strs = []
    strs.append('\\begin{table*}[t]\n')
    strs.append('\\caption{' + title + '}\n')
    strs.append('\\label{' + label + '}\n')
    strs.append('\\centering\n')
    #strs.append('\\small\n')
    strs.append('\\begin{tabular}{l|' + (mean_df.shape[1])*'c' + '} \\hline\n')
    for col in mean_df.columns:
        strs.append(' & ' + col)
    strs.append(' \\\\ \\hline\n')
    for ix in mean_df.index:
        for j in range(mean_df.shape[1]):
            if j == 0: strs.append(ix + ' ')
            jx = mean_df.columns[j]
            strs.append('& $')
            if ismax_df.loc[ix, jx] == True: strs.append('\\mathbf{')
            if ismax2nd_df.loc[ix, jx] == True: strs.append('\\mathit{')
            strs.append('%.3f' % mean_df.loc[ix, jx] + '\\pm' + '%.3f' % std_df.loc[ix, jx])
            if ismax_df.loc[ix, jx] == True or ismax2nd_df.loc[ix, jx] == True: strs.append('}')
            strs.append('$ ')
        strs.append('\\\\\n')
    strs.append('\\hline\n')
    strs.append('\\end{tabular}\n')
    strs.append('\\end{table*}\n')
    return ''.join(strs)

Generate evaluation data tables.

In [ ]:
func = calc_F1
#func = calc_pairsF1

In [ ]:
#if func == calc_Tau: 
#    methods = [methods_all[0]] + methods_all[2:]
#else:
#    methods = methods_all.copy()   
methods = methods_all.copy() 

for type_ix in range(len(dat_types)):
    mean_df = pd.DataFrame(data=np.zeros((len(methods), len(dat_names)), dtype=np.float), \
                           columns=dat_names, index=methods)
    std_df  = pd.DataFrame(data=np.zeros((len(methods), len(dat_names)), dtype=np.float), \
                           columns=dat_names, index=methods)
        
    for dat_ix in range(len(dat_suffix)):
        recdict_rank, recdict_tran, recdict_comb, recdict_rand, recdict_ijcai = load_results(dat_ix, type_ix)
        means, stds = calc_metrics(recdict_rank, recdict_tran, recdict_comb, recdict_rand, recdict_ijcai, func)
        #print(len(means), len(stds), len(methods))
        assert(len(means) == len(stds) == len(methods))
        mean_df[dat_names[dat_ix]] = means
        std_df[dat_names[dat_ix]]  = stds
        
    ismax_df  = pd.DataFrame(data=np.zeros(mean_df.shape, dtype=np.bool), columns=mean_df.columns, index=mean_df.index)
    ismax2nd_df = ismax_df.copy()
    for col in ismax_df.columns:
        #maxix = mean_df[col].argmax()
        #ismax_df.loc[maxix, col] = True
        indices = (-mean_df[col]).argsort().values[:2]
        ismax_df.iloc[indices[0]][col] = True
        ismax2nd_df.iloc[indices[1]][col] = True
    
    if func == calc_F1:
        title = '''Performance comparison on five datasets in terms of trajectory F$_1$ score. 
        The best method for each dataset (i.e., a column) is shown in bold, the second best is shown in italic.'''
        label = 'tab:f1'
    else:
        title = '''Performance comparison on five datasets in terms of pairs-F$_1$.
        The best method for each dataset (i.e., a column) is shown in bold, the second best is shown in italic.'''
        label = 'tab:pairf1'
    strs = gen_latex_table(mean_df, std_df, ismax_df, ismax2nd_df, type_ix, title, label)
    #mean_df.to_csv('pf1_new.csv')
    
    print(strs)

Compute the difference between the new performance table and the old one.

In [ ]:
#F1new = pd.read_csv('f1_new.csv')
#F1new.drop(F1new.columns[0], axis=1, inplace=True)
#F1new.head()

In [ ]:
#F1old = pd.read_csv('f1_old.csv')
#F1old.drop(F1old.columns[0], axis=1, inplace=True)
#F1old.head()

In [ ]:
#F1diff = pd.DataFrame(data=(F1new-F1old).values, columns=dat_names, index=methods_all)
#zero_df = pd.DataFrame(data=np.zeros(F1diff.shape), columns=dat_names, index=methods_all)

In [ ]:
#print(gen_latex_table(F1diff, zero_df, zero_df, zero_df, 0, uspecific, 'F1 improvement', 'tab:f1'))

In [ ]:
#pF1new = pd.read_csv('pf1_new.csv')
#pF1new.drop(pF1new.columns[0], axis=1, inplace=True)
#pF1new.head()

In [ ]:
#pF1old = pd.read_csv('pf1_old.csv')
#pF1old.drop(pF1old.columns[0], axis=1, inplace=True)
#pF1old.head()

In [ ]:
#pF1diff = pd.DataFrame(data=(pF1new-pF1old).values, columns=dat_names, index=methods_all)
#zero_df = pd.DataFrame(data=np.zeros(pF1diff.shape), columns=dat_names, index=methods_all)

In [ ]:
#print(gen_latex_table(pF1diff, zero_df, zero_df, zero_df, 0, uspecific, 'pairs-F1 improvement', 'tab:pf1'))